In [1]:
import scipy.io as scio
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
import time
from os.path import dirname, join as pjoin
from sklearn import preprocessing
import pickle
from scipy.io import loadmat
from scipy.io import savemat
from matplotlib.pyplot import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, '能用')

cuda 能用


In [3]:
file_H='1D S-V\1D S-V data.mat'
dataf_H=loadmat(file_H,mat_dtype=True)
dataf_H=dataf_H['Hrecord']
data_H = np.transpose(np.array(dataf_H[15000:,1:202])) # [feature, time_step]

In [4]:
def make_LSTM_datasets(data,lookback,shift_k,train_size):
    #train_size = 2000
    val_size = 500
    samples = train_size + val_size
    n_features = data.shape[0]
    sdata = data[:,shift_k:shift_k+samples]

    Xtemp = {}
    for i in range(lookback):
        Xtemp[i] = sdata[:,i:samples-(lookback-i)]

    X = Xtemp[0]
    for i in range(lookback-1):
        X = np.vstack([X,Xtemp[i+1]])
    
    X = torch.tensor(X)
    X = X.t()
    Y = np.transpose(sdata[:,lookback:])
    Y=torch.tensor(Y)

    Xtrain = X[:train_size-(lookback),:].to(torch.float32)
    Ytrain = Y[:train_size-(lookback),:].to(torch.float32)

    Xval = X[train_size-(lookback):, :].to(torch.float32)
    Yval = Y[train_size-(lookback):, :].to(torch.float32)

    Xtrain = Xtrain.reshape((Xtrain.shape[0], lookback, n_features))
    Xval =  Xval.reshape((Xval.shape[0], lookback, n_features))
    print("Xtrain shape = ", Xtrain.shape, "Ytrain shape = ", Ytrain.shape)
    print("Xval shape =   ", Xval.shape, "  Yval shape =   ", Yval.shape)

    return Xtrain, Ytrain, Xval, Yval

In [5]:
class RNNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layer, device):
        super(RNNLSTM, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.device = device
        self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=self.hidden_dim,
                            num_layers=self.n_layer, batch_first=True)
        self.fc = nn.Linear(in_features=self.hidden_dim, out_features=self.input_dim)

    def forward(self, x):
        lstm_outputs, state = self.lstm(x)
        last = lstm_outputs[:, -1, :]
        prediction = self.fc(last)
        return prediction, state

In [6]:
def MakeTrainLoader(Xtrain, Ytrain):
    train_dataset = TensorDataset(Xtrain, Ytrain)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=20,
        shuffle=True,
        num_workers=2
    )
    return train_loader

In [7]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    train_loss = []
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        ypred,state = model(x)
        loss = criterion(ypred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    train_loss = torch.tensor(train_loss)
    return torch.mean(train_loss)

In [8]:
def validate(Xval, Yval, model, lookback, device, criterion):
    model.eval()
    Xval = Xval.to(device)
    Yval = Yval.to(device)
    ypred = torch.zeros((Xval.shape[0], Yval.shape[1])).to(device)
    loss = torch.zeros((Xval.shape[0]))
    for i in range(Xval.shape[0]):
        if i==0:
            tt = Xval[0,:,:].reshape((1,lookback,Yval.shape[1]))
            ypred[i,:],state = model(tt)
        elif i<lookback:
            tt = Xval[i,:,:].reshape((1,lookback,Yval.shape[1]))
            u = ypred[:i,:]
            tt[0,(lookback-i):lookback,:] = u
            ypred[i, :], state = model(tt)
        else:
            tt = ypred[i-lookback:i,:].reshape((1,lookback,Yval.shape[1]))
            ypred[i,:],state = model(tt)
        loss[i] = criterion(ypred[i,:],Yval[i,:])
    val_loss = criterion(ypred, Yval)
    return  ypred, val_loss, loss

In [9]:
def computeRMSE(X_pred, X_true):
    X_pred_avg = X_pred[:,:,:]
    X_true_avg = X_true[:,:,:]
    err = X_true_avg-X_pred_avg
    err_temp = torch.multiply(err, err)
    RMSE_ini = torch.sqrt(torch.mean(err_temp, dim=1))  # [time_step,ic]
    RMSE_time = torch.mean(RMSE_ini, dim=1)  # [time_step]
    return RMSE_ini, RMSE_time

In [67]:
def computeAnomalyCorrelationCoefficient(X_pred, X_true):
    AC_vec = torch.zeros((X_pred.shape[0], X_pred.shape[2]))  # X_pred[time_step,feature,ic(initial condition)]
    for ic in range(X_pred.shape[2]):
        X_pred_avg = X_pred[:,:,ic]
        X_true_avg = X_true[:,:,ic]
        X_true_mean = torch.mean(X_true[:,:,ic], dim=0)  # real mean in whole time [feature]
        temp1 = (X_pred_avg-X_true_mean)*(X_true_avg-X_true_mean)
        temp1 = torch.sum(temp1, dim=1)  # [time_step]
        temp2 = (X_pred_avg-X_true_mean)*(X_pred_avg-X_true_mean)
        temp2 = torch.sqrt(torch.sum(temp2, dim=1))  # [time_step]
        temp3 = (X_true_avg-X_true_mean)*(X_true_avg-X_true_mean)
        temp3 = torch.sqrt(torch.sum(temp3, dim=1))  # [time_step]
        AC_vec[:,ic] = temp1/(temp2*temp3)  # [time_step,ic]
    print("Mean anomaly correlation coefficient over"+str(X_pred.shape[2])
          + " different initial conditions")
    AC = torch.mean(AC_vec, dim=1)  # [time_step]
    return AC, AC_vec

In [10]:
lookback= 4
hidden_dim=800
n_layer=2
lr = 0.001
epoch=50
criterion = nn.MSELoss().to(device)  # 损失函数

In [81]:
Y_true=torch.zeros((500,200,28)) # [time_step,feature,ic(initial condition)]
Y_pred=torch.zeros((500,200,28))
Y_val_loss_H=torch.zeros((500,28))

In [82]:
for i in range(28):
    shift_k=3000*i
    print(i,shift_k)
    HXtrain, HYtrain, HXval, HYval = make_LSTM_datasets(data_H, lookback, shift_k)
    modelH = RNNLSTM(input_dim=200, hidden_dim=hidden_dim, n_layer=n_layer,\
                 device=device).to(device)
    optimizerH = torch.optim.Adam(modelH.parameters(), lr)
    for e in range(50):
        train_loader_H = MakeTrainLoader(HXtrain, HYtrain)
        train_loss_H = train(train_loader_H, modelH, criterion, optimizerH, device)
        print("epoch",e,"train loss",train_loss_H)
    val_ypred_H, val_loss_H, loss_H = validate(HXval, HYval, modelH, lookback,\
                                           device, criterion)
    Y_true[:,:,i]=HYval
    Y_pred[:,:,i]=val_ypred_H
    Y_val_loss_H[:,i]=val_loss_H

0 0
Xtrain shape =  torch.Size([1996, 4, 200]) Ytrain shape =  torch.Size([1996, 200])
Xval shape =    torch.Size([500, 4, 200])   Yval shape =    torch.Size([500, 200])
epoch 0 train loss tensor(0.0319)
epoch 1 train loss tensor(0.0098)
epoch 2 train loss tensor(0.0043)
epoch 3 train loss tensor(0.0020)
epoch 4 train loss tensor(0.0013)
epoch 5 train loss tensor(0.0008)
epoch 6 train loss tensor(0.0006)
epoch 7 train loss tensor(0.0006)
epoch 8 train loss tensor(0.0005)
epoch 9 train loss tensor(0.0005)
epoch 10 train loss tensor(0.0004)
epoch 11 train loss tensor(0.0004)
epoch 12 train loss tensor(0.0004)
epoch 13 train loss tensor(0.0004)
epoch 14 train loss tensor(0.0004)
epoch 15 train loss tensor(0.0002)
epoch 16 train loss tensor(0.0002)
epoch 17 train loss tensor(0.0002)
epoch 18 train loss tensor(0.0001)
epoch 19 train loss tensor(0.0001)
epoch 20 train loss tensor(0.0001)
epoch 21 train loss tensor(0.0001)
epoch 22 train loss tensor(0.0002)
epoch 23 train loss tensor(0.0001)


epoch 3 train loss tensor(0.0029)
epoch 4 train loss tensor(0.0029)
epoch 5 train loss tensor(0.0023)
epoch 6 train loss tensor(0.0010)
epoch 7 train loss tensor(0.0007)
epoch 8 train loss tensor(0.0007)
epoch 9 train loss tensor(0.0005)
epoch 10 train loss tensor(0.0004)
epoch 11 train loss tensor(0.0004)
epoch 12 train loss tensor(0.0003)
epoch 13 train loss tensor(0.0003)
epoch 14 train loss tensor(0.0003)
epoch 15 train loss tensor(0.0003)
epoch 16 train loss tensor(0.0002)
epoch 17 train loss tensor(0.0002)
epoch 18 train loss tensor(0.0002)
epoch 19 train loss tensor(0.0002)
epoch 20 train loss tensor(0.0002)
epoch 21 train loss tensor(0.0002)
epoch 22 train loss tensor(0.0002)
epoch 23 train loss tensor(0.0002)
epoch 24 train loss tensor(0.0001)
epoch 25 train loss tensor(0.0001)
epoch 26 train loss tensor(0.0001)
epoch 27 train loss tensor(0.0001)
epoch 28 train loss tensor(0.0001)
epoch 29 train loss tensor(8.6494e-05)
epoch 30 train loss tensor(0.0001)
epoch 31 train loss ten

epoch 12 train loss tensor(0.0005)
epoch 13 train loss tensor(0.0004)
epoch 14 train loss tensor(0.0003)
epoch 15 train loss tensor(0.0003)
epoch 16 train loss tensor(0.0002)
epoch 17 train loss tensor(0.0002)
epoch 18 train loss tensor(0.0002)
epoch 19 train loss tensor(0.0002)
epoch 20 train loss tensor(0.0002)
epoch 21 train loss tensor(0.0002)
epoch 22 train loss tensor(0.0002)
epoch 23 train loss tensor(0.0002)
epoch 24 train loss tensor(0.0002)
epoch 25 train loss tensor(0.0001)
epoch 26 train loss tensor(0.0001)
epoch 27 train loss tensor(0.0001)
epoch 28 train loss tensor(0.0001)
epoch 29 train loss tensor(0.0001)
epoch 30 train loss tensor(0.0001)
epoch 31 train loss tensor(8.6071e-05)
epoch 32 train loss tensor(7.9232e-05)
epoch 33 train loss tensor(0.0001)
epoch 34 train loss tensor(0.0001)
epoch 35 train loss tensor(0.0001)
epoch 36 train loss tensor(7.6421e-05)
epoch 37 train loss tensor(8.2401e-05)
epoch 38 train loss tensor(7.7486e-05)
epoch 39 train loss tensor(8.9597e-

epoch 18 train loss tensor(0.0003)
epoch 19 train loss tensor(0.0002)
epoch 20 train loss tensor(0.0003)
epoch 21 train loss tensor(0.0002)
epoch 22 train loss tensor(0.0002)
epoch 23 train loss tensor(0.0002)
epoch 24 train loss tensor(0.0002)
epoch 25 train loss tensor(0.0002)
epoch 26 train loss tensor(0.0002)
epoch 27 train loss tensor(0.0001)
epoch 28 train loss tensor(0.0001)
epoch 29 train loss tensor(0.0001)
epoch 30 train loss tensor(0.0001)
epoch 31 train loss tensor(9.5312e-05)
epoch 32 train loss tensor(9.7093e-05)
epoch 33 train loss tensor(0.0001)
epoch 34 train loss tensor(0.0001)
epoch 35 train loss tensor(8.0075e-05)
epoch 36 train loss tensor(0.0001)
epoch 37 train loss tensor(7.8029e-05)
epoch 38 train loss tensor(0.0001)
epoch 39 train loss tensor(6.6641e-05)
epoch 40 train loss tensor(5.8629e-05)
epoch 41 train loss tensor(7.2834e-05)
epoch 42 train loss tensor(7.5655e-05)
epoch 43 train loss tensor(6.0559e-05)
epoch 44 train loss tensor(7.9103e-05)
epoch 45 train 

epoch 25 train loss tensor(0.0001)
epoch 26 train loss tensor(8.9495e-05)
epoch 27 train loss tensor(7.2244e-05)
epoch 28 train loss tensor(6.4534e-05)
epoch 29 train loss tensor(7.8883e-05)
epoch 30 train loss tensor(6.6859e-05)
epoch 31 train loss tensor(9.1870e-05)
epoch 32 train loss tensor(5.5341e-05)
epoch 33 train loss tensor(6.2004e-05)
epoch 34 train loss tensor(0.0002)
epoch 35 train loss tensor(6.5160e-05)
epoch 36 train loss tensor(4.7288e-05)
epoch 37 train loss tensor(4.0769e-05)
epoch 38 train loss tensor(4.4025e-05)
epoch 39 train loss tensor(4.4799e-05)
epoch 40 train loss tensor(5.9428e-05)
epoch 41 train loss tensor(4.0790e-05)
epoch 42 train loss tensor(6.9697e-05)
epoch 43 train loss tensor(3.7049e-05)
epoch 44 train loss tensor(5.1168e-05)
epoch 45 train loss tensor(4.0562e-05)
epoch 46 train loss tensor(3.0962e-05)
epoch 47 train loss tensor(6.7225e-05)
epoch 48 train loss tensor(4.6834e-05)
epoch 49 train loss tensor(3.8114e-05)
17 51000
Xtrain shape =  torch.Si

epoch 31 train loss tensor(0.0001)
epoch 32 train loss tensor(9.0939e-05)
epoch 33 train loss tensor(8.0510e-05)
epoch 34 train loss tensor(7.2983e-05)
epoch 35 train loss tensor(9.9888e-05)
epoch 36 train loss tensor(7.9436e-05)
epoch 37 train loss tensor(7.5715e-05)
epoch 38 train loss tensor(7.9981e-05)
epoch 39 train loss tensor(7.5798e-05)
epoch 40 train loss tensor(8.1078e-05)
epoch 41 train loss tensor(6.2162e-05)
epoch 42 train loss tensor(7.4847e-05)
epoch 43 train loss tensor(7.6914e-05)
epoch 44 train loss tensor(6.2769e-05)
epoch 45 train loss tensor(5.6513e-05)
epoch 46 train loss tensor(5.9628e-05)
epoch 47 train loss tensor(9.8360e-05)
epoch 48 train loss tensor(5.6481e-05)
epoch 49 train loss tensor(5.4230e-05)
21 63000
Xtrain shape =  torch.Size([1996, 4, 200]) Ytrain shape =  torch.Size([1996, 200])
Xval shape =    torch.Size([500, 4, 200])   Yval shape =    torch.Size([500, 200])
epoch 0 train loss tensor(0.0216)
epoch 1 train loss tensor(0.0004)
epoch 2 train loss t

epoch 39 train loss tensor(7.8280e-05)
epoch 40 train loss tensor(7.1810e-05)
epoch 41 train loss tensor(8.5497e-05)
epoch 42 train loss tensor(7.0468e-05)
epoch 43 train loss tensor(7.3778e-05)
epoch 44 train loss tensor(7.0580e-05)
epoch 45 train loss tensor(5.8012e-05)
epoch 46 train loss tensor(5.5621e-05)
epoch 47 train loss tensor(6.0752e-05)
epoch 48 train loss tensor(5.0591e-05)
epoch 49 train loss tensor(8.1511e-05)
25 75000
Xtrain shape =  torch.Size([1996, 4, 200]) Ytrain shape =  torch.Size([1996, 200])
Xval shape =    torch.Size([500, 4, 200])   Yval shape =    torch.Size([500, 200])
epoch 0 train loss tensor(0.0219)
epoch 1 train loss tensor(0.0005)
epoch 2 train loss tensor(0.0005)
epoch 3 train loss tensor(0.0005)
epoch 4 train loss tensor(0.0005)
epoch 5 train loss tensor(0.0005)
epoch 6 train loss tensor(0.0005)
epoch 7 train loss tensor(0.0005)
epoch 8 train loss tensor(0.0005)
epoch 9 train loss tensor(0.0005)
epoch 10 train loss tensor(0.0005)
epoch 11 train loss t

In [83]:
RMSE_ini,RMSE_time=computeRMSE(Y_pred, Y_true)
AC, AC_vec=computeAnomalyCorrelationCoefficient(Y_pred, Y_true)

Mean anomaly correlation coefficient over28 different initial conditions


In [84]:
savemat('lookback'+str(lookback)+'_hd'+str(hidden_dim)+'_nl'+str(n_layer)+'_Shift_28'+'.mat',\
        {'Y_true':Y_true.detach().numpy(),'Y_pred':Y_pred.detach().numpy(),'RMSE_ini':RMSE_ini.detach().numpy(),\
         'RMSE_time':RMSE_time.detach().numpy(),'AC':AC.detach().numpy(),'AC_vec':AC_vec.detach().numpy(),\
         'Y_val_loss_H':Y_val_loss_H.detach().numpy()
    
})